In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv drive/MyDrive/kidneydataset.zip .

In [ ]:
! unzip /content/kidneydataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal/Normal- (4363).jpg  
  inflating: CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal/Normal- (4364).jpg  
  inflating: CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal/Normal- (4365).jpg  
  inflating: CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal/Normal- (4366).jpg  
  inflating: CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal/Normal- (4367).jpg  
  inflating: CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal/Normal- (4368).jpg  
  inflating: CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal/Normal- (4369).jpg  
  inflating: CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-N

In [ ]:
!mv CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone data

In [ ]:
!rm -r data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Cyst

In [ ]:
!rm -r data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Tumor

In [ ]:
!rm kidneyData.csv

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.xception import preprocess_input

def extract_features_from_folders(folder_paths):

    base_model = Xception(weights='imagenet', include_top=False)

    all_features = []
    for folder_path in folder_paths:
        features_list = []
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(folder_path, filename)
                features = extract_features(image_path, base_model)
                features_list.append(features)
        all_features.append(features_list)
    return all_features

def extract_features(image_path, model):
    # Load and preprocess the image
    img = keras_image.load_img(image_path, target_size=(128, 128))
    img_array = keras_image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)


    features = model.predict(img_array)
    features = np.ravel(features)
    return features


folder_paths = ['/content/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal',
                '/content/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone']
all_features = extract_features_from_folders(folder_paths)

for i, features_list in enumerate(all_features):
    print("Features for images in folder", i+1)
    print("Number of images:", len(features_list))
    if len(features_list) > 0:
        print("Dimensions of each image:", len(features_list[0]))


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 94ms/step
Features for images in folder 1
Number of images: 5077
Dimensions of each image: 32768
Features for images in folder 2
Number of images: 1377
Dimensions of each image: 32768


In [ ]:
import os
import shutil
import random

def split_data(source, train_dir, test_dir, split_ratio):
    for class_folder in os.listdir(source):
        class_path = os.path.join(source, class_folder)
        if os.path.isdir(class_path):
            files = os.listdir(class_path)
            random.shuffle(files)  # Shuffle the files to ensure randomness in the split
            split_index = int(len(files) * split_ratio)
            train_files = files[:split_index]
            test_files = files[split_index:]

            # Create class folders in train and test directories if they don't exist
            train_class_dir = os.path.join(train_dir, class_folder)
            test_class_dir = os.path.join(test_dir, class_folder)
            os.makedirs(train_class_dir, exist_ok=True)
            os.makedirs(test_class_dir, exist_ok=True)

            # Move files to train directory
            for file in train_files:
                src = os.path.join(class_path, file)
                dst = os.path.join(train_class_dir, file)
                shutil.copy(src, dst)

            # Move files to test directory
            for file in test_files:
                src = os.path.join(class_path, file)
                dst = os.path.join(test_class_dir, file)
                shutil.copy(src, dst)

In [ ]:
source_directory = "data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
train_dir = "Train"
test_dir = "Test"
split_ratio = 0.8  #(80% training data, 20% testing data)

In [ ]:
split_data(source_directory, train_dir, test_dir, split_ratio)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Image dimensions
img_height = 256
img_width = 256
batch_size = 32

In [ ]:
# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load and prepare the dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 5162 images belonging to 2 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 1292 images belonging to 2 classes.


In [ ]:
# CNN model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=4,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size)

Epoch 1/4
161/161 [==============================] - 925s 6s/step - loss: 0.2535 - accuracy: 0.8979 - val_loss: 0.1736 - val_accuracy: 0.9352
Epoch 2/4
161/161 [==============================] - 911s 6s/step - loss: 0.0899 - accuracy: 0.9692 - val_loss: 0.0228 - val_accuracy: 0.9945
Epoch 3/4
161/161 [==============================] - 889s 6s/step - loss: 0.0530 - accuracy: 0.9819 - val_loss: 0.0192 - val_accuracy: 0.9969
Epoch 4/4
161/161 [==============================] - 911s 6s/step - loss: 0.0334 - accuracy: 0.9879 - val_loss: 0.0772 - val_accuracy: 0.9664


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
accuracy_percent = round(accuracy * 100, 2)
print("Test Accuracy:", accuracy)
print("Test Accuracy:", accuracy_percent, "%")

41/41 [==============================] - 61s 1s/step - loss: 0.0770 - accuracy: 0.9667
Test Accuracy: 0.9667182564735413
Test Accuracy: 96.67 %


In [ ]:
# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(train_generator)
train_accuracy_percent = round(train_accuracy * 100, 2)
print("Training Accuracy:",train_accuracy_percent)
print("Training Accuracy:", train_accuracy_percent, "%")


162/162 [==============================] - 321s 2s/step - loss: 0.0773 - accuracy: 0.9684
Training Accuracy: 96.84
Training Accuracy: 96.84 %


In [ ]:
# Save the trained model
model.save('kidney_stone_detection_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import files
files.download('kidney_stone_detection_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
model = tf.keras.models.load_model('kidney_stone_detection_model.h5')

In [ ]:
# Define function to preprocess image
def preprocess_image(image_path, target_size=(256, 256)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    return img_array

In [ ]:
# Load and preprocess the image
image_path = '/content/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone/Stone- (1).jpg'
img = preprocess_image(image_path)

In [ ]:
prediction = model.predict(img)

1/1 [==============================] - 0s 146ms/step


In [ ]:
print(prediction[0][0])

0.9946065


In [ ]:
# Output prediction
if prediction[0][0] > 0.5:
    print("The image contains a kidney stone.")
else:
    print("The image does not contain a kidney stone.")

The image contains a kidney stone.


In [ ]:
import cv2
import numpy as np
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def segment_kidney_stones(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    circular_contours = []
    min_area = 100  #noise
    min_circularity = 0.7
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_area:
            continue
        perimeter = cv2.arcLength(contour, True)
        circularity = 4 * np.pi * area / (perimeter * perimeter)
        if circularity > min_circularity:
            circular_contours.append(contour)
    sizes = []
    for contour in circular_contours:
        area = cv2.contourArea(contour)
        diameter = np.sqrt(4 * area / np.pi)
        sizes.append((area, diameter))
    return sizes

image_path = '/content/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone/Stone- (1).jpg'
thresh = preprocess_image(image_path)
sizes = segment_kidney_stones(thresh)
if sizes:
    print("Circular kidney stones detected in", image_path)
    for i, (area, diameter) in enumerate(sizes, 1):
        area_rounded = round(area)
        diameter_rounded = round(diameter)
        print(f"Kidney Stone {i}: Area={area_rounded} pixels, Diameter={diameter_rounded} mm")
else:
    print("No circular kidney stones detected in", image_path)


Circular kidney stones detected in /content/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone/Stone- (1).jpg
Kidney Stone 1: Area=400 pixels, Diameter=23 mm


# **Position Of The Stone**

In [ ]:
import cv2
import numpy as np

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def segment_kidney_stones(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    circular_contours = []
    min_area = 100  #noise
    min_circularity = 0.7
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_area:
            continue
        perimeter = cv2.arcLength(contour, True)
        circularity = 4 * np.pi * area / (perimeter * perimeter)
        if circularity > min_circularity:
            circular_contours.append(contour)

    sizes_and_centroids = []
    for contour in circular_contours:
        area = cv2.contourArea(contour)
        diameter = np.sqrt(4 * area / np.pi)
        moments = cv2.moments(contour)
        if moments['m00'] != 0:
            centroid_x = int(moments['m10'] / moments['m00'])
            centroid_y = int(moments['m01'] / moments['m00'])
            centroid = (centroid_x, centroid_y)
        else:
            centroid = None
        sizes_and_centroids.append((area, diameter, centroid))

    return sizes_and_centroids

image_path = '/content/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone/Stone- (1).jpg'
thresh = preprocess_image(image_path)
sizes_and_centroids = segment_kidney_stones(thresh)

if sizes_and_centroids:
    print("Circular kidney stones detected in", image_path)
    for i, (area, diameter, centroid) in enumerate(sizes_and_centroids, 1):
        area_rounded = round(area)
        diameter_rounded = round(diameter)
        centroid_text = f"Centroid(X,Y-axis): {centroid}" if centroid else "Centroid: Not available"
        print(f"Kidney Stone {i}: Diameter={diameter_rounded} mm, {centroid_text}")
else:
    print("No circular kidney stones detected in", image_path)


Circular kidney stones detected in /content/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone/Stone- (1).jpg
Kidney Stone 1: Diameter=23 mm, Centroid(X,Y-axis): (169, 180)


In [ ]:
import cv2
import numpy as np

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def segment_kidney_stones(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    circular_contours = []
    min_area = 100  #noise
    min_circularity = 0.7
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_area:
            continue
        perimeter = cv2.arcLength(contour, True)
        circularity = 4 * np.pi * area / (perimeter * perimeter)
        if circularity > min_circularity:
            circular_contours.append(contour)

    sizes_and_centroids = []
    for contour in circular_contours:
        area = cv2.contourArea(contour)
        diameter = np.sqrt(4 * area / np.pi)
        moments = cv2.moments(contour)
        if moments['m00'] != 0:
            centroid_x = int(moments['m10'] / moments['m00'])
            centroid_y = int(moments['m01'] / moments['m00'])
            centroid = (centroid_x, centroid_y)
        else:
            centroid = None
        sizes_and_centroids.append((area, diameter, centroid))

    return sizes_and_centroids

def recommend_treatment(diameter, pass_threshold, surgery_threshold):
    if diameter < pass_threshold:
        return "Pass on its own"
    elif diameter < surgery_threshold:
        return "Medication"
    else:
        return "Surgery"

def main():
    image_path = '/content/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone/Stone- (1).jpg'
    thresh = preprocess_image(image_path)
    sizes_and_centroids = segment_kidney_stones(thresh)

    pass_threshold = 5
    surgery_threshold = 30

    if sizes_and_centroids:
        print("Circular kidney stones detected in", image_path)
        for i, (area, diameter, centroid) in enumerate(sizes_and_centroids, 1):
            area_rounded = round(area)
            diameter_rounded = round(diameter)
            centroid_text = f"Centroid(X,Y-axis): {centroid}" if centroid else "Centroid: Not available"
            print(f"Kidney Stone {i}:\nDiameter={diameter_rounded} mm \n{centroid_text}")
            treatment = recommend_treatment(diameter_rounded, pass_threshold, surgery_threshold)
            print(f"Treatment Recommendation: {treatment}")
    else:
        print("No circular kidney stones detected in", image_path)

if __name__ == "__main__":
    main()


Circular kidney stones detected in /content/data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone/Stone- (1).jpg
Kidney Stone 1:
Diameter=23 mm 
Centroid(X,Y-axis): (169, 180)
Treatment Recommendation: Medication


UI

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from PIL import Image
import cv2
import numpy as np

# Set background color
background_color = "#f0f0f0"  # Light gray

# Set background image
background_image = "/content/drive/MyDrive/Image1.jpeg"  # Path to your background image

# Set CSS styles for the app with background color and image
st.markdown(
    f"""
    <style>
    .reportview-container {{
        background-color: {background_color};
        background-image: url('/content/drive/MyDrive/Image1.jpeg');
        background-size: cover;
    }}
    </style>
    """,
    unsafe_allow_html=True
)



2024-05-17 07:27:18.570 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.141.147.173


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.141.147.173:8501

npx: installed 22 in 2.359s
your url is: https://mean-paws-enjoy.loca.lt
  Stopping...
^C
